In [1]:
import pandas as pd
import requests
from datetime import datetime, timedelta

In [2]:
import pandas as pd
import requests
from datetime import datetime, timedelta

def date_query(start_date, end_date):
    #formats date and makes sure everything will be in datetime
    date_format = "%Y-%m-%d"
    start = datetime.strptime(start_date, date_format)
    end = datetime.strptime(end_date, date_format)
    delta = timedelta(days=1)
    #stores all the dataframes from the query
    frames = []
    #goes through each day one by one and grabs the dataframes
    while start <= end:
        date_str = start.strftime(date_format)
        url = f"https://www.sofascore.com/api/v1/sport/football/scheduled-events/{date_str}"
        response = requests.get(url)

        if response.status_code == 200:
            data = response.json()
            events = data.get('events', [])

            if events:
                df = pd.json_normalize(events)
                df['Date'] = date_str

                # Select specific columns
                selected_columns = ['homeTeam.name', 'awayTeam.name', 'id', 'homeScore.current', 'awayScore.current',
                                    'tournament.name', 'season.id', 'tournament.id', 'Date']

                df_selected = df[selected_columns]

                # Rename columns for clarity
                df_selected.columns = ['Home Team', 'Away Team', 'Event ID', 'Home Score', 'Away Score',
                                       'Tournament Name', 'Season ID', 'Tournament ID', 'Date']

                frames.append(df_selected)
        else:
            print(f"Failed to fetch data for {date_str}. Status code: {response.status_code}")

        start += delta

    if frames:
        result_df = pd.concat(frames, ignore_index=True)
        return result_df
    else:
        return pd.DataFrame()  # Return an empty DataFrame if no data was fetched

# Example usage
df_result = date_query('2024-05-19', '2024-05-21')
df_result

,Home Team,Away Team,Event ID,Home Score,Away Score,Tournament Name,Season ID,Tournament ID,Date
0,Arsenal,Everton,11352546,2.0,1.0,Premier League,52186,1,2024-05-19
1,Brentford,Newcastle United,11352549,3.0,5.0,Premier League,52186,1,2024-05-19
2,Brighton & Hove Albion,Manchester United,11352552,1.0,2.0,Premier League,52186,1,2024-05-19
3,Burnley,Nottingham Forest,11352555,2.0,3.0,Premier League,52186,1,2024-05-19
4,Chelsea,Bournemouth,11352558,2.0,2.0,Premier League,52186,1,2024-05-19
...,...,...,...,...,...,...,...,...,...
1198,Khánh Hoà,Thể Công - Viettel,12217417,0.0,2.0,V-League 1,55929,4104,2024-05-21
1199,MerryLand Quy Nhơn Bình Định FC,Sông Lam Nghệ An,12219803,1.0,2.0,V-League 1,55929,4104,2024-05-21
1200,Nam Định,Hải Phòng,12217418,2.0,4.0,V-League 1,55929,4104,2024-05-21
1201,Tottenham Hotspur,Newcastle United,12268514,6.0,6.0,Club Friendly Games,57234,86,2024-05-21
